# 🧠 AI Agents Bootcamp: Running DeepSeek with CrewAI and Ollama
This notebook is part of the AI Agents Bootcamp (23–27 June 2025) — it shows how to:
- Set up the `Ollama` environment in Colab
- Run `DeepSeek` models locally for use in CrewAI/MAS pipelines
- Prepare agents that use locally-hosted LLMs with memory and tools

## ⚙️ Step 1: Environment Setup
This installs and runs the `ollama` backend and exposes the service via localtunnel tunnel. Make sure to:
- Restart the runtime if needed
- Use the ngrok alternative (if Cloudflare is blocked or throttled)

In [ ]:
%pip install ollama
%pip install colab-xterm

## 🛠️ System Info Tools (Optional)

Installs utilities (`pciutils`, `lshw`) to inspect hardware specs — useful for checking GPU/CPU availability in Colab or local runtime.








In [ ]:
!sudo apt-get update
!sudo apt-get install pciutils lshw

## 📦 Ollama Installation

Downloads and installs Ollama via the official shell script — run this once per environment setup.

In [ ]:
!curl -fsSL https://ollama.com/install.sh | sh

## 🔧 Step 2: Programmatic Model Management and Server Initialization

In this section, we:
- Import the required libraries for managing subprocesses, HTTP requests, and multithreading
- Start the Ollama server programmatically using a background thread
- Pull the required models (`deepseek-r1:7b`, `llama3`) using `ollama pull`
- Optionally include fallback to a smaller model (`deepseek-r1:1.5b`)
- Confirm the list of available models and test that the local Ollama server is running at `localhost:11434`

📌 **Why it matters**: This sets up your local model infrastructure for agent interaction. You'll later reference `localhost:11434` in your agent definitions to connect to these models.


In [ ]:
# Import necessary libraries
import subprocess
import requests
import json
import threading
from pprint import pprint

##  Launching the Ollama Server in Background

Before using any model, we need to start the **Ollama inference server**, which listens by default on `localhost:11434`.

This snippet:
- Defines a Python function `run_ollama()` that launches `ollama serve`
- Starts it in a **background thread**, so the notebook remains interactive
- Allows the server to stay active without blocking further cells

🛠️ **Note**: You only need to run this once per session. If you restart your Colab, re-run this cell before using any models.


In [ ]:
# Start the Ollama server
def run_ollama():
  subprocess.Popen(["ollama", "serve"])
thread = threading.Thread(target=run_ollama)
thread.start()

## 📥 Pulling Models

We download pre-trained models from the Ollama registry:
- `deepseek-r1:7b` – reasoning & code
- `llama3` – general-purpose assistant

In [ ]:
# Download the deepseek-r1:7b distilled model
!ollama pull deepseek-r1:7b
!ollama pull llama3
# If this doesn't work, you can uncomment the below code to download a smaller model- deepseek-r1:1.5b
# !ollama pull deepseek-r1:1.5b

## 🪶 Pulling Lightweight SLMs

These small models are ideal for fast local agents and low-resource environments:
- `phi3:mini`, `tinyllama` – ultra-small general models
- `gemma:2b` – Google's compact chat model
- `deepseek-r1:1.5b` – distilled reasoning model

In [ ]:
!ollama pull phi3:mini
!ollama pull tinyllama
!ollama pull gemma:2b
!ollama pull deepseek-r1:1.5b

## 🔌 Test Ollama Server

Sends a test request to verify the Ollama server is running on `localhost:11434`.

In [ ]:
!curl http://127.0.0.1:11434

## 📄 Check Installed Models

Lists all models currently downloaded and available in your local Ollama environment.

In [ ]:
!ollama list

# 🧠 Starting the CrewAI Section

##Now we define agents using CrewAI, connected to our locally running Ollama models.  
##This enables multi-agent workflows powered by lightweight, self-hosted LLMs.


In [ ]:
# @title 👨‍🦯 Run this cell to hide all warnings (optional)
# Warning control
import warnings
warnings.filterwarnings('ignore')

# To avoid the restart session warning in Colab, exclude the PIL and
# pydevd_plugins packages from being imported. This is fine because
# we didn't execute the code in the kernel session afterward.

# import sys
# sys.modules.pop('PIL', None)

In [ ]:
# @title ⬇️ Install project dependencies by running this cell
%pip install git+https://github.com/joaomdmoura/crewAI.git --quiet
%pip install crewai_tools langchain_openai langchain_groq langchain_anthropic langchain_community cohere --quiet
print("---")
%pip show crewAI crewai_tools langchain_openai langchain_groq langchain_anthropic langchain_community cohere

In [ ]:
%pip install -qU langchain-ollama

## 🧩 Step 3: CrewAI Integration


In [ ]:
# imports


import os
from getpass import getpass
from crewai import Agent, Task, Crew, Process
from textwrap import dedent
from langchain_ollama import ChatOllama


## Define Agents
In CrewAI, agents are autonomous entities designed to perform specific roles and achieve particular goals. Each agent uses a language model (LLM) and may have specialized tools to help execute tasks.

In [ ]:
# @title 🕵🏻 Define your agents

from crewai import Agent
from textwrap import dedent
from langchain_ollama import ChatOllama
# Define LLM (OpenAI used here; replace as needed)
from crewai import LLM

llm = LLM(model="ollama/tinyllama:latest", base_url="http://127.0.0.1:11434")

# Agent 1: Researcher
agent_1 = Agent(
    role=dedent("""Researcher"""),
    goal=dedent("""Identify reliable, up-to-date information on a technical topic."""),
    backstory=dedent("""You are an experienced research assistant skilled at finding credible sources, synthesizing data, and distilling it into key insights."""),
    allow_delegation=False,
    verbose=True,
    max_iter=3,
    llm=llm
)

# Agent 2: Analyst
agent_2 = Agent(
    role=dedent("""Analyst"""),
    goal=dedent("""Interpret research findings and draw meaningful conclusions."""),
    backstory=dedent("""You are a data-driven thinker who turns raw information into structured, actionable insights. You are logical and detail-oriented."""),
    allow_delegation=False,
    verbose=True,
    max_iter=3,
    llm=llm
)

# Agent 3: Writer
agent_3 = Agent(
    role=dedent("""Technical Writer"""),
    goal=dedent("""Craft a well-structured, engaging article based on the insights provided by the team."""),
    backstory=dedent("""You are a clear and persuasive communicator who turns complex ideas into accessible writing for both technical and general audiences."""),
    allow_delegation=False,
    verbose=True,
    max_iter=3,
    llm=llm
)


## Define Tasks
Tasks in CrewAI are specific assignments given to agents, detailing the actions they need to perform to achieve a particular goal. Tasks can have dependencies and context, and can be executed asynchronously to ensure an efficient workflow.

In [ ]:
# @title 📝 Define your tasks
from crewai import Task
from textwrap import dedent

# Task 1: Researcher
task_1 = Task(
    description=dedent("""
        Research recent developments in open-source large language models (LLMs), with a focus on DeepSeek, its capabilities, and recent benchmarks. Identify 2–3 major advantages compared to closed-source models.
    """),
    expected_output=dedent("""
        A structured summary (~200 words) covering:
        - What DeepSeek is and who developed it
        - Key features or innovations
        - At least two comparative strengths of open-source LLMs
    """),
    agent=agent_1,
)

# Task 2: Analyst
task_2 = Task(
    description=dedent("""
        Analyze the research findings about DeepSeek and extract educational implications. Focus on how this technology could benefit educators, students, or institutions using local infrastructure.
    """),
    expected_output=dedent("""
        A short analytical breakdown (~150 words) including:
        - 2–3 practical use cases in education
        - The impact of offline/local LLMs on cost and data privacy
        - Any challenges or limitations to note
    """),
    agent=agent_2,
    context=[task_1],
)

# Task 3: Writer
task_3 = Task(
    description=dedent("""
        Write a clear, engaging blog post summarizing the findings and analysis into a cohesive article titled:
        "How Open-Source AI Like DeepSeek Is Changing Education."
    """),
    expected_output=dedent("""
        A 400–500 word markdown-formatted blog post including:
        - Title and subtitle
        - Introduction
        - Three core paragraphs with bullet points if needed
        - Conclusion and future outlook
    """),
    agent=agent_3,
    context=[task_2],
)


In [ ]:
print("## 👥 Welcome to the DeepSeek Research Crew")
print('-------------------------------------------')

# Input variables for tasks
var_1 = input("🔍 Topic or model to explore (e.g., DeepSeek)?\n")
var_2 = input("🎯 Who is the target audience (e.g., educators, developers)?\n")
var_3 = input("🧠 What’s the intended output format (e.g., article, summary)?\n")

print('-------------------------------------------')
print(f"✔️ Input received:\nModel: {var_1}\nAudience: {var_2}\nOutput Type: {var_3}")

In [ ]:
# @title 🚀 Get your crew to work!
def main():
    # Instantiate your crew with a sequential process
    crew = Crew(
        agents=[agent_1, agent_2, agent_3],
        tasks=[task_1, task_2, task_3],
        verbose=True,  # You can set it to True or False
        # ↑ indicates the verbosity level for logging during execution.
        process=Process.sequential,
        planning_llm=llm
        # ↑ the process flow that the crew will follow (e.g., sequential, hierarchical).
    )

    inputs = {
    "var_1": var_1,
    "var_2": var_2,
    "var_3": var_3
    }

    result = crew.kickoff(inputs=inputs)
    print("\n\n########################")
    print("## Here is your custom crew run result:")
    print("########################\n")
    print(result)

    return result

if __name__ == "__main__":
  result = main()

In [ ]:
# @title 🖥️ Display the results of your crew as markdown
from IPython.display import display, Markdown

markdown_text = result.raw  # Adjust this based on the actual attribute

# Display the markdown content
display(Markdown(markdown_text))